In [1]:
# This notebook is by Anastasia Ruzmaikina. It uses Llama-2-7b in Kaggle Competition US Patent Phrase to Phrase Matching

Can you extract meaning from a large, text-based dataset derived from inventions? Here's your chance to do so.

The U.S. Patent and Trademark Office (USPTO) offers one of the largest repositories of scientific, technical, and commercial information in the world through its Open Data Portal. Patents are a form of intellectual property granted in exchange for the public disclosure of new and useful inventions. Because patents undergo an intensive vetting process prior to grant, and because the history of U.S. innovation spans over two centuries and 11 million patents, the U.S. patent archives stand as a rare combination of data volume, quality, and diversity.

In this competition, you will train your models on a novel semantic similarity dataset to extract relevant information by matching key phrases in patent documents. Determining the semantic similarity between phrases is critically important during the patent search and examination process to determine if an invention has been described before. For example, if one invention claims "television set" and a prior publication describes "TV set", a model would ideally recognize these are the same and assist a patent attorney or examiner in retrieving relevant documents. This extends beyond paraphrase identification; if one invention claims a "strong material" and another uses "steel", that may also be a match. What counts as a "strong material" varies per domain (it may be steel in one domain and ripstop fabric in another, but you wouldn't want your parachute made of steel). We have included the Cooperative Patent Classification as the technical domain context as an additional feature to help you disambiguate these situations.

Can you build a model to match phrases in order to extract contextual information, thereby helping the patent community connect the dots between millions of patent documents?



In this notebook I use Llama-2-7b to classify key phrases appearing in patents by similarity to known phrases. The classifier outputs the similarity scores 0, 0.25, 0.5, 0.75, 1.

This notebook has 61.5% accuracy on the competition test dataset. A random guessing algorithm would have 20% accuracy since there are five possible scores. Increasing accuracy of Llama2 would require to use larger r-value (here I used r=4) and therefore increased training time.

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/us-patent-phrase-to-phrase-matching/sample_submission.csv
/kaggle/input/us-patent-phrase-to-phrase-matching/train.csv
/kaggle/input/us-patent-phrase-to-phrase-matching/test.csv
/kaggle/input/accelerate-and-bitsandbytes/accelerate-0.29.3-py3-none-any.whl
/kaggle/input/accelerate-and-bitsandbytes/bitsandbytes-0.43.1-py3-none-manylinux_2_24_x86_64.whl
/kaggle/input/daigt-pip/peft-0.6.0-py3-none-any.whl
/kaggle/input/daigt-pip/optimum-1.14.0-py3-none-any.whl
/kaggle/input/daigt-pip/tokenizers-0.14.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
/kaggle/input/daigt-pip/transformers-4.35.0-py3-none-any.whl
/kaggle/input/huggingface-hub-permissible-models-and-datasets/top_300_permissible_models.csv
/kaggle/input/huggingface-hub-permissible-models-and-datasets/all_datasets.csv
/kaggle/input/huggingface-hub-permissible-models-and-datasets/all_models.csv
/kaggle/input/huggingface-hub-permissible-models-and-datasets/all_spaces.csv
/kaggle/input/d/jefersonpazze/llama2-7b

In [3]:
# Install package for inferences
!pip install -qq --no-deps /kaggle/input/daigt-pip/peft-0.6.0-py3-none-any.whl --use-deprecated=legacy-resolver
!pip install -qq --no-deps /kaggle/input/daigt-pip/transformers-4.35.0-py3-none-any.whl --use-deprecated=legacy-resolver
!pip install -qq --no-deps /kaggle/input/daigt-pip/tokenizers-0.14.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl --use-deprecated=legacy-resolver
!pip install -qq --no-deps /kaggle/input/accelerate-and-bitsandbytes/accelerate-0.29.3-py3-none-any.whl --use-deprecated=legacy-resolver
!pip install -qq --no-deps /kaggle/input/accelerate-and-bitsandbytes/bitsandbytes-0.43.1-py3-none-manylinux_2_24_x86_64.whl --use-deprecated=legacy-resolver
!pip install -qq --no-deps /kaggle/input/daigt-pip/optimum-1.14.0-py3-none-any.whl --use-deprecated=legacy-resolver
#!pip install -qq --no-deps /kaggle/input/llm-detect-pip/accelerate-0.24.1-py3-none-any.whl
#!pip install -qq --no-deps /kaggle/input/llm-detect-pip/bitsandbytes-0.41.1-py3-none-any.whl
#!os.register_at_fork()

In [4]:
from __future__ import annotations

TARGET_MODEL = '/kaggle/input/llama2-7b-hf/Llama2-7b-hf'#"mistralai/Mistral-7B-v0.1"

DEBUG = False
from pathlib import Path

OUTPUT_DIR = Path("./")
OUTPUT_DIR.mkdir(exist_ok=True, parents=True)

INPUT_DIR = Path("../input/")


In [5]:
#!pip install -q -U peft --no-index --find-links ../input/llm-detect-pip/ --use-deprecated=legacy-resolver
#!pip install -q -U accelerate --no-index --find-links ../input/llm-detect-pip/ --use-deprecated=legacy-resolver
#!pip install -q -U bitsandbytes --no-index --find-links ../input/llm-detect-pip/ --use-deprecated=legacy-resolver
#!pip install -q -U transformers --no-index --find-links ../input/llm-detect-pip/ --use-deprecated=legacy-resolver
# Install package for inferences
#!pip install -qq --no-deps /kaggle/input/hf-libraries/bitsandbytes --use-deprecated=legacy-resolver
!pip install -qq --no-deps /kaggle/input/daigt-pip/peft-0.6.0-py3-none-any.whl --use-deprecated=legacy-resolver
#!pip install -qq --no-deps /kaggle/input/daigt-pip/transformers-4.35.0-py3-none-any.whl 
#!pip install -qq --no-deps /kaggle/input/daigt-pip/tokenizers-0.14.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!pip install -qq --no-deps /kaggle/input/daigt-pip/optimum-1.14.0-py3-none-any.whl --use-deprecated=legacy-resolver
#!pip install -qq --no-deps /kaggle/input/llm-detect-pip/accelerate-0.24.1-py3-none-any.whl
#!pip install -qq --no-deps /kaggle/input/llm-detect-pip/bitsandbytes-0.41.1-py3-none-any.whl --use-deprecated=legacy-resolver
#!pip install -qq --no-deps /kaggle/input/bitsandbytes-0-42-0 --use-deprecated=legacy-resolver

ERROR: Directory '/kaggle/input/bitsandbytes-0-42-0' is not installable. Neither 'setup.py' nor 'pyproject.toml' found.


In [6]:
import pandas as pd
train_df = pd.read_csv('/kaggle/input/us-patent-phrase-to-phrase-matching/train.csv')#(INPUT_DIR / "", sep=',')
train_df.score = train_df['score'] * 4
train_df.score = train_df['score'].round().astype(int)
train_df['input'] = 'ANCHOR: ' + train_df.anchor + '; TARGET:' + train_df.target + ';'
train_df.insert(1, 'input', train_df.pop('input'))
train_df = train_df.sample(frac=1)
train_df = train_df.reset_index()

test_df = pd.read_csv('/kaggle/input/us-patent-phrase-to-phrase-matching/test.csv')  #(INPUT_DIR / "", sep=',')
test_df['input'] = 'ANCHOR: ' + test_df.anchor + '; TARGET:' + test_df.target + ';'
test_df.insert(1, 'input', test_df.pop('input'))

train_df

,index,id,input,anchor,target,context,score
0,24642,6b3767533645a402,ANCHOR: polls; TARGET:grips;,polls,grips,B21,3
1,4308,3ecb239abc30accd,ANCHOR: carburization; TARGET:ferrous metals;,carburization,ferrous metals,B23,1
2,8562,b383be364933dd71,ANCHOR: define panel; TARGET:define internal p...,define panel,define internal panel,B65,2
3,10335,7f2fc70fdfa18c87,ANCHOR: dual clutch; TARGET:dual clutch in tra...,dual clutch,dual clutch in tractor,B62,2
4,15399,37e4b7aba2d2455b,ANCHOR: her2 targeted; TARGET:receptor tyrosin...,her2 targeted,receptor tyrosine protein kinase erbb 2,A61,2
...,...,...,...,...,...,...,...
36468,8614,ad60c58d0705710d,ANCHOR: deflect light; TARGET:electric bulb;,deflect light,electric bulb,H01,1
36469,13035,b85f0f63384bb17c,ANCHOR: fit within perimeter; TARGET:enclosed ...,fit within perimeter,enclosed perimeter,B01,3
36470,11959,b5aa182bf28bbb90,ANCHOR: estimated amount; TARGET:list;,estimated amount,list,G06,0
36471,26953,75f7cf6c79b16f4d,ANCHOR: rear adjustment; TARGET:rear movement;,rear adjustment,rear movement,E05,3


In [7]:
train_df = train_df.rename(columns={'score': 'label'})
train_df

,index,id,input,anchor,target,context,label
0,24642,6b3767533645a402,ANCHOR: polls; TARGET:grips;,polls,grips,B21,3
1,4308,3ecb239abc30accd,ANCHOR: carburization; TARGET:ferrous metals;,carburization,ferrous metals,B23,1
2,8562,b383be364933dd71,ANCHOR: define panel; TARGET:define internal p...,define panel,define internal panel,B65,2
3,10335,7f2fc70fdfa18c87,ANCHOR: dual clutch; TARGET:dual clutch in tra...,dual clutch,dual clutch in tractor,B62,2
4,15399,37e4b7aba2d2455b,ANCHOR: her2 targeted; TARGET:receptor tyrosin...,her2 targeted,receptor tyrosine protein kinase erbb 2,A61,2
...,...,...,...,...,...,...,...
36468,8614,ad60c58d0705710d,ANCHOR: deflect light; TARGET:electric bulb;,deflect light,electric bulb,H01,1
36469,13035,b85f0f63384bb17c,ANCHOR: fit within perimeter; TARGET:enclosed ...,fit within perimeter,enclosed perimeter,B01,3
36470,11959,b5aa182bf28bbb90,ANCHOR: estimated amount; TARGET:list;,estimated amount,list,G06,0
36471,26953,75f7cf6c79b16f4d,ANCHOR: rear adjustment; TARGET:rear movement;,rear adjustment,rear movement,E05,3


In [8]:
train_df.label.value_counts()

label
2    12300
1    11519
0     7471
3     4029
4     1154
Name: count, dtype: int64

In [9]:
train_df = train_df.drop(['id', 'target', 'anchor'], axis=1)
test1_df = test_df
test_df = test_df.drop(['id', 'target', 'anchor'], axis=1)
train_df.reset_index(inplace=True, drop=True)
print(f"Train dataframe has shape: {train_df.shape}")
train_df.head()

Train dataframe has shape: (36473, 4)


,index,input,context,label
0,24642,ANCHOR: polls; TARGET:grips;,B21,3
1,4308,ANCHOR: carburization; TARGET:ferrous metals;,B23,1
2,8562,ANCHOR: define panel; TARGET:define internal p...,B65,2
3,10335,ANCHOR: dual clutch; TARGET:dual clutch in tra...,B62,2
4,15399,ANCHOR: her2 targeted; TARGET:receptor tyrosin...,A61,2


In [10]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=20, shuffle=True, random_state=42)
X = train_df.loc[:, train_df.columns != "label"]
y = train_df.loc[:, train_df.columns == "label"]

for i, (train_index, valid_index) in enumerate(skf.split(X, y)):
    train_df.loc[valid_index, "fold"] = i
    
print(train_df.groupby("fold")["label"].value_counts())
train_df.head()

fold  label
0.0   2        615
      1        576
      0        373
      3        202
      4         58
              ... 
19.0  2        615
      1        576
      0        373
      3        201
      4         58
Name: count, Length: 100, dtype: int64


,index,input,context,label,fold
0,24642,ANCHOR: polls; TARGET:grips;,B21,3,5.0
1,4308,ANCHOR: carburization; TARGET:ferrous metals;,B23,1,7.0
2,8562,ANCHOR: define panel; TARGET:define internal p...,B65,2,11.0
3,10335,ANCHOR: dual clutch; TARGET:dual clutch in tra...,B62,2,2.0
4,15399,ANCHOR: her2 targeted; TARGET:receptor tyrosin...,A61,2,14.0


In [11]:
# fold0 as valid
valid_df = train_df[train_df["fold"] == 0]
train_df = train_df[train_df["fold"] != 0]
print(train_df.shape)
print(valid_df.shape)

(34649, 5)
(1824, 5)


In [12]:
# load model with 4bit bnb

from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType # type: ignore
from transformers import BitsAndBytesConfig
import torch

peft_config = LoraConfig(
    r=4,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.SEQ_CLS,
    inference_mode=False,
    target_modules=[
        "q_proj",
        "v_proj"
    ],
)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

/opt/conda/lib/python3.10/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/opt/conda/lib/python3.10/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [13]:
!python -m bitsandbytes

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
++++++++++++++++++ BUG REPORT INFORMATION ++++++++++++++++++
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
++++++++++++++++++++++++++ OTHER +++++++++++++++++++++++++++
CUDA specs: CUDASpecs(highest_compute_capability=(6, 0), cuda_version_string='123', cuda_version_tuple=(12, 3))
PyTorch settings found: CUDA_VERSION=123, Highest Compute Capability: (6, 0).
To manually override the PyTorch CUDA version please see: https://github.com/TimDettmers/bitsandbytes/blob/main/docs/source/nonpytorchcuda.mdx
If you run into issues with 8-bit matmul, you can try 4-bit quantization:
https://huggingface.co/blog/4bit-transformers-bitsandbytes
Found duplicate CUDA runtime files (see below).

We select the PyTorch default CUDA runtime, which is 12.3,
but this might mismatch with the CUDA version that is needed for bitsandbytes.
To override this behavior set the `BNB_CUDA_VERSION=<version string, e.g. 122>` environmental variable.

In [14]:
from transformers import AutoTokenizer, LlamaForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained(TARGET_MODEL, use_fast=False)
tokenizer.pad_token = tokenizer.eos_token

In [15]:
#!pip install accelerate
#!pip install bitsandbytes>=0.39.0

In [16]:
base_model = LlamaForSequenceClassification.from_pretrained(
    TARGET_MODEL,
    num_labels=5,
    quantization_config=bnb_config,
    device_map={"":0}
)
base_model.config.pretraining_tp = 1 # 1 is 7b
base_model.config.pad_token_id = tokenizer.pad_token_id

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/transformers/modeling_utils.py:484: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkpoint_file, map_location=map

In [17]:
model = get_peft_model(base_model, peft_config)


In [18]:
model.print_trainable_parameters()


trainable params: 2,117,632 || all params: 6,609,481,728 || trainable%: 0.03203930485243638


In [19]:
#remove this for actual calculation  here we take a smaller sample of the dataframe to speed up 
train_df = train_df.sample(2000, random_state=42)  #2400


In [20]:
valid_df1 = valid_df.sample(frac = 0.4, random_state=42)
valid_df = valid_df.loc[~valid_df.index.isin(valid_df1.index)]
print(train_df.shape)
print(valid_df.shape)
print(valid_df1.shape)

(2000, 5)
(1094, 5)
(730, 5)


In [21]:
# datasets
from datasets import Dataset

# from pandas
train_ds = Dataset.from_pandas(train_df)
valid_ds = Dataset.from_pandas(valid_df)
test_ds = Dataset.from_pandas(test_df)
valid_ds1 = Dataset.from_pandas(valid_df1)

In [22]:
def preprocess_function(examples, max_length=512):
    return tokenizer(examples["input"], truncation=True, max_length=max_length, padding=True)

In [23]:
train_tokenized_ds = train_ds.map(preprocess_function, batched=True)
valid_tokenized_ds = valid_ds.map(preprocess_function, batched=True)
test_tokenized_ds = test_ds.map(preprocess_function, batched=True)
valid_tokenized_ds1 = valid_ds1.map(preprocess_function, batched=True)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1094 [00:00<?, ? examples/s]

Map:   0%|          | 0/36 [00:00<?, ? examples/s]

Map:   0%|          | 0/730 [00:00<?, ? examples/s]

In [24]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding="longest")

In [25]:
import numpy as np
from sklearn.metrics import accuracy_score, roc_auc_score
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    
    accuracy_val = accuracy_score(labels, predictions)
    #roc_auc_val = roc_auc_score(labels, predictions)
    
    return {
        "accuracy": accuracy_val,
        #"roc_auc": roc_auc_val,
    }

In [26]:
from transformers import TrainingArguments, Trainer

steps = 5 if DEBUG else 20

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    learning_rate=5e-4,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=16,
    max_grad_norm=0.3,
    optim='paged_adamw_32bit',
    lr_scheduler_type="cosine",
    num_train_epochs=1,
    weight_decay=0.01,
    evaluation_strategy="steps",
    save_strategy="steps",
    load_best_model_at_end=True,
    push_to_hub=False,
    warmup_steps=steps,
    eval_steps=steps,
    logging_steps=steps,
    report_to='none' # if DEBUG else 'wandb',
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized_ds,
    eval_dataset=valid_tokenized_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
20,1.647300,1.537109,0.368373
40,1.470700,1.559570,0.394881
60,1.305500,1.213867,0.534735
80,1.195000,1.213867,0.446984
100,1.128400,1.025391,0.575868
120,1.006400,1.001953,0.582267


TrainOutput(global_step=125, training_loss=1.2838131103515624, metrics={'train_runtime': 5376.65, 'train_samples_per_second': 0.372, 'train_steps_per_second': 0.023, 'total_flos': 2643191169024000.0, 'train_loss': 1.2838131103515624, 'epoch': 1.0})

In [27]:
#this is to check if the predictions work on a known dataframe not previously seen by the model
preds = trainer.predict(valid_tokenized_ds1)
logits = preds.predictions

In [28]:
# from scipy.special import expit as sigmoid
import numpy as np
def sigmoid(x):
    return 1 / (1 + np.exp(-x))  
logits = sigmoid(logits)
logits = np.argmax(logits, axis=1)
logits[:100]



array([1, 2, 0, 3, 3, 2, 2, 2, 1, 0, 2, 0, 1, 2, 1, 2, 2, 1, 2, 1, 2, 0,
       2, 1, 2, 2, 1, 2, 2, 0, 2, 1, 1, 2, 2, 1, 1, 0, 0, 2, 0, 3, 2, 0,
       1, 2, 0, 1, 2, 1, 3, 1, 2, 2, 2, 2, 2, 2, 2, 0, 1, 1, 2, 1, 1, 1,
       0, 1, 3, 2, 2, 2, 1, 0, 0, 2, 0, 2, 2, 1, 2, 1, 1, 2, 2, 0, 1, 3,
       4, 1, 1, 1, 1, 3, 2, 1, 1, 0, 1, 2])

In [29]:
#this is to check if the predictions work on a known dataframe not previosly seen by the model
valid_df1['preds'] = logits
valid_df1

,index,input,context,label,fold,preds
11198,24689,ANCHOR: polyglycolic acid; TARGET:acid;,C04,1,0.0,1
34693,23100,ANCHOR: oxygen carrier; TARGET:carrier for oxy...,F23,4,0.0,2
5658,34480,ANCHOR: use as cooling fluid; TARGET:thermometer;,F24,0,0.0,0
14934,30843,ANCHOR: slip segment; TARGET:slip element;,H01,3,0.0,3
18573,33226,ANCHOR: tooth spaces; TARGET:tooth gaps;,G01,3,0.0,3
...,...,...,...,...,...,...
29914,24232,ANCHOR: physical transport medium; TARGET:phys...,G06,2,0.0,3
23123,19945,ANCHOR: melt coating; TARGET:melt coating;,B41,4,0.0,4
8209,2079,ANCHOR: arcuate means; TARGET:guide members;,D03,1,0.0,1
1879,12550,ANCHOR: faucet assembly; TARGET:faucet;,F16,2,0.0,2


In [30]:
#count = valid_df1[(valid_df1['preds'] > 0.8 ) & (valid_df1['label'] == 4)].shape[0] + valid_df1[0.6 <(valid_df1['preds'] <= 0.8  ) & (valid_df1['label'] == 3)].shape[0]+ valid_df1[0.4 <(valid_df1['preds'] <= 0.6 ) & (valid_df1['label'] == 2)].shape[0]+ valid_df1[0.2 <(valid_df1['preds'] <= 0.4  ) & (valid_df1['label'] == 1)].shape[0]+ valid_df1[0 =<(valid_df1['preds'] <= 0.2  ) & (valid_df1['label'] == 0)].shape[0]
#print(count, count/valid_df1.shape[0])

In [31]:
preds = trainer.predict(test_tokenized_ds)#.predictions.astype(float)
logits = preds.predictions


In [32]:
# from scipy.special import expit as sigmoid
import numpy as np
def sigmoid(x):
    #if x > -100:
        return 1 / (1 + np.exp(-x))  
   # else:
       # return 0
logits = sigmoid(logits)
logits = np.argmax(logits, axis=1)
logits[:100]

array([2, 2, 1, 1, 2, 2, 2, 0, 1, 2, 1, 1, 2, 3, 3, 1, 1, 1, 2, 1, 2, 1,
       2, 1, 2, 0, 0, 0, 0, 3, 0, 1, 3, 1, 1, 1])

In [35]:
#this is to provide the sample submission
sub = pd.DataFrame()
test_df = pd.read_csv('/kaggle/input/us-patent-phrase-to-phrase-matching/test.csv')
#sub['id'] = valid_df['id']
sub['id'] = test_df['id']
import math
sub['score'] = logits
#sub['score'] = sub['score'].round(2)
sub['score'] = sub['score'].apply(lambda x: 0.00 if x==0 else x)
sub['score'] = sub['score'].apply(lambda x: 0.25 if x==1 else x)
sub['score'] = sub['score'].apply(lambda x: 0.50 if x==2 else x)
sub['score'] = sub['score'].apply(lambda x: 0.75 if x==3 else x)
sub['score'] = sub['score'].apply(lambda x: 1.00 if x==4 else x)

sub.to_csv('/kaggle/working/submission.csv', index=False)
sub.head()

,id,score
0,4112d61851461f60,0.50
1,09e418c93a776564,0.50
2,36baf228038e314b,0.25
3,1f37ead645e7f0c8,0.25
4,71a5b6ad068d531f,0.50


In [36]:
dfs = pd.read_csv('/kaggle/working/submission.csv')
dfs

,id,score
0,4112d61851461f60,0.50
1,09e418c93a776564,0.50
2,36baf228038e314b,0.25
3,1f37ead645e7f0c8,0.25
4,71a5b6ad068d531f,0.50
5,474c874d0c07bd21,0.50
6,442c114ed5c4e3c9,0.50
7,b8ae62ea5e1d8bdb,0.00
8,faaddaf8fcba8a3f,0.25
9,ae0262c02566d2ce,0.50
